In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp

import healpy as hp
import pointing_groups as pg
import requests

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.io.votable import parse
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs import NoConvergence

from queryMPC import runMPCRequests
#from queryMPC import matchSingleVisit
from plottingTools import makeStamps

pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [83]:
with open('A0b.pkl', 'rb') as f:
    PointingGroups = pickle.load(f)
PointingGroups.drop_duplicates('visit_id',inplace=True)
df = PointingGroups
date_obs = df['date_obs'].iloc[0].decode()#[2:-1]
time_obj = Time(date_obs, format='iso', scale='utc')
time_obj.jd

2458576.714648935

In [73]:
def createObsTable(df):
    """Create the input to the MPC post request.
    
    Input-
        df : pandas dataframe
            The dataframe of observations for a particular request.
    
    Output-
        textarea : string
            The string that should be the 'textarea' value in the payload dictionary
    """
    
    textarea = ''
    
    for idx in range(len(df)):
        # Convert the dataframe ra and dec into Sky Coordinates
        c = SkyCoord(df['ra'].iloc[idx]*u.degree, df['dec'].iloc[idx]*u.degree)
        # Convert RA and DEC into hour-minute-second and degree-minute-second
        ra_hms = c.ra.hms
        dec_dms = c.dec.dms
        # Get the observation time and convert it into a standard format
        date_obs = df['date_obs'].iloc[idx].decode()#[2:-1]
        time_obj = Time(date_obs, format='iso', scale='utc')
        # Convert observation time and sky coords into a string
        if dec_dms.d != 0:
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%+03i %02i %05.2f                     W84\n" %
                        (int(df['visit_id'].iloc[idx]), date_obs[:4], date_obs[5:7], 
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms.d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        else:
            if copysign(1, dec_dms.d) == -1.0:
                dec_dms_d = '-00'
            else:
                dec_dms_d = '+00'
            name = ("     %07i   %s %s %s.%s %02i %02i %06.3f%s %02i %05.2f                     W84\n" %
                        (df['visit_id'].iloc[idx], date_obs[:4], date_obs[5:7],
                         date_obs[8:10], str(time_obj.mjd)[6:11],
                         ra_hms.h, ra_hms.m, ra_hms.s,
                         dec_dms_d, np.abs(dec_dms.m), np.abs(dec_dms.s)))
        textarea += name
        
    return textarea

def parseResults(result):
    """Parse the results of the MPC requests query.
    Input-
        result : requests http query result
    Output-
        results_df : pandas dataframe containing results for the query
    """
    # Split the results based on newline characters
    results_cut = result.text.split('\n')[12:-49][6:-1]
    # Initialize lists of the values to be parsed from results_cut 
    visit_id = []
    name = []
    ra_hour = []
    ra_min = []
    ra_sec = []
    dec_deg = []
    dec_min = []
    dec_sec = []
    v_mag = []
    ra_motion = []
    dec_motion = []
    # Iterate through results_cut and append them to the respective lists
    for line in results_cut:
        try:
            ra_hour.append(int(line[24:27]))
            ra_min.append(int(line[28:30]))
            ra_sec.append(float(line[31:35]))
            dec_deg.append(int(line[35:39]))
            dec_min.append(int(line[39:42]))
            dec_sec.append(int(line[42:45]))
            v_mag.append(float(line[46:51]))
            ra_motion.append('%s%i' % (line[71], int(line[68:71])))
            dec_motion.append('%s%i' % (line[78], int(line[75:78])))
            visit_id.append(str(line[0:8]))
            name.append(line[8:22])
        except:
            # If there is no reported value for the object, return -99
            visit_id.append(str(-99.0))
            name.append(-99.0)
            ra_hour.append(-99.0)
            ra_min.append(-99.0)
            ra_sec.append(-99.0)
            dec_deg.append(-99.0)
            dec_min.append(-99.0)
            dec_sec.append(-99.0)
            v_mag.append(-99.0)
            ra_motion.append(99.0)
            dec_motion.append(99.0)
 
    results_df = pd.DataFrame()
    results_df['visit_id'] = visit_id
    results_df['name'] = name
    results_df['ra_hour'] = ra_hour
    results_df['ra_min'] = ra_min
    results_df['ra_sec'] = ra_sec
    results_df['dec_deg'] = dec_deg
    results_df['dec_min'] = dec_min
    results_df['dec_sec'] = dec_sec
    results_df['v_mag'] = v_mag
    results_df['ra_motion'] = np.array(ra_motion).astype(float)
    results_df['dec_motion'] = np.array(dec_motion).astype(float)
    
    return results_df

url     = 'https://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi'
headers = {}
# Set the list of returns from MPC
payload = {'year':'2018',
           'month':'01',
           'day': '16.67',
           'which':'obs',
           'ra':'',
           'decl':'',
           'TextArea':'',
           'radius':'90',
           'limit':'30.0',
           'oc':'500',
           'sort':'d',
           'mot':'h',
           'tmot':'s',
           'pdes':'u',
           'needed':'f',
           'ps':'n',
           'type':'p'
          }
# Initialize a results dataframe 
results_df = None
# Generate a table of observation times and sky coordinates from dataframe
ta = createObsTable(PointingGroups[0:1])+createObsTable(PointingGroups[-1:])
payload['TextArea'] = ta
# Print table for user verification
print(ta)
# Generate the request for MPC
res = requests.post(url, data=payload, headers=headers)
# Add the requests to results_df, appending if it exists
if results_df is None:
    results_df = parseResults(res)
    results_df['field'] = 'A0b'
else:
    label_results_df = parseResults(res)
    label_results_df['field'] = 'A0b'
    results_df = results_df.append(label_results_df, ignore_index=True)
#print(results_df)

     0845580   2019 04 03.21464 14 23 27.300-13 36 31.20                     W84
     0845682   2019 04 03.39042 14 23 26.050-13 36 29.30                     W84



In [74]:
results_df

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field
0,(370485),2003 QQ67,14.0,23.0,28.2,-13.0,33.0,16.0,20.7,-24.0,14.0,A0b
1,(160199),2002 AJ45,14.0,23.0,11.1,-13.0,36.0,50.0,20.8,-22.0,9.0,A0b
2,(96460),1998 HJ36,14.0,23.0,43.8,-13.0,29.0,52.0,18.7,-28.0,-2.0,A0b
3,,2018 CH6,14.0,23.0,17.3,-13.0,28.0,42.0,21.4,-26.0,11.0,A0b
4,(541055),2018 CH6,14.0,23.0,17.3,-13.0,28.0,42.0,21.4,-26.0,11.0,A0b
5,(422726),2001 FX140,14.0,23.0,45.8,-13.0,28.0,9.0,19.9,-21.0,0.0,A0b
6,(334282),2001 UR134,14.0,22.0,48.8,-13.0,43.0,39.0,21.5,-27.0,6.0,A0b
7,(253647),2003 UD129,14.0,23.0,19.1,-13.0,48.0,38.0,20.4,-25.0,16.0,A0b
8,,2018 AY,14.0,23.0,26.5,-13.0,48.0,49.0,29.0,-29.0,9.0,A0b
9,(214629),2006 RO98,14.0,22.0,38.5,-13.0,41.0,12.0,20.8,-28.0,-0.0,A0b


In [75]:
# Initialize ccd, x_pixel, and y_pixel values
# This prepares the dataframe for the pixel matching
results_df['ccd'] = -99
results_df['x_pixel'] = -99
results_df['y_pixel'] = -99
# Calculate total motion
results_df['total_motion'] = np.sqrt(results_df['ra_motion']**2. + results_df['dec_motion']**2.)
len(results_df)

1175

In [76]:
# Match the objects from MPC with pixel locations in the DECam NEO Survey
# Can comment this out if you don't want to select on slow movers (total_motion is arcsec/hr.)
# Cut the dataframe based on desired brightness and speed
resultsDF = results_df
cutDF = resultsDF.query('v_mag > 21 and total_motion < 20')
cutDF = cutDF.reset_index(drop=True)
# Generate a list of visits in the pointing group
#nightVisits = np.unique(cutDF['visit_id'])
nightVisits = np.unique(['0'+visit.decode('UTF-8') for visit in PointingGroups['visit_id']])
# Generate the path to the data
dataPath = '/astro/store/pogo4/smotherh/DEEP/PointingGroups/A0b/warps'

def matchPixels(visit):
    """This is a wrapper for matchSingleVisit.
    It allows it to easily run in parallel with a simple
    pool.map() call.
    """
    #visitDF = cutDF.query('visit_id == %i' % visit)
    #visitDF = visitDF.reset_index(drop=True)
    visitDF = matchSingleVisit(cutDF,visit,dataPath)
    print('Processed visit {}'.format(visit))
    return(visitDF)

def matchSingleVisit(visitDF,visit,dataPath,verbose=False):
    """Match the MPC object RA and DEC to pixel coorinates in the DECam NEO Survey
    Inputs-
        visitDF : Dataframe from runMPCRequests.
            It should only contain values from a single visit, although it may
            have multiple objects in that visit.
            Can be cut on magnitude, etc. if desired
        dataPath : Path to the DECam NEO Survey warps.
        verbose : Verbosity flag for print output

    Outputs-
        visitDF : Updated Dataframe with object->pixel relationships.
    """    
    if verbose:
        print('Processing visit {}.'.format(visit))
    # Iterate over all moving objects (rows) in the visitDF
    for idx, obj_row in visitDF.iterrows():
        # Set ra and dec and use them to generate a SkyCoord object
        ra = '%i:%i:%.1f' % (obj_row['ra_hour'], obj_row['ra_min'], obj_row['ra_sec'])
        dec = '%i:%i:%.1f' % (obj_row['dec_deg'], obj_row['dec_min'], obj_row['dec_sec'])
        c = SkyCoord(ra, dec, frame='icrs', unit=(u.hourangle, u.deg))
        # Iterate over CCDs
        for i in [43]:#range(1, 63):
            # CCD 2 and 61 are broken on DECam and should be skipped
            if (i!=43):
                continue
            if verbose:
                print('Processing ccd {} of 62.'.format(i))
            # Calculate the pixel values for the objects in the visit
            try:
                # Load only the fits header, changing the path for varying CCDs
                fitsHeader = fits.getheader('%s/%02i/%s.fits' % (dataPath,i,visit),1)
                # Load the world coordinate system and find the pixel values
                w = WCS(fitsHeader)
                x_pix, y_pix = c.to_pixel(w)
                # If the returned pixel values are on the given CCD, save the object
                if (x_pix < 2010) and (x_pix > 0) and (y_pix < 4100) and (y_pix > 0):
                    if verbose:
                        print(obj_row['name'], ra, dec)
                        print(x_pix, y_pix)
                    visitDF['x_pixel'].iloc[idx] = x_pix
                    visitDF['y_pixel'].iloc[idx] = y_pix
                    visitDF['ccd'].iloc[idx] = i
                    visitDF['visit_id'].iloc[idx] = visit
                    break
            except NoConvergence:
                continue
    return(visitDF)

with mp.Pool(20) as pool:
    results = pool.map(matchPixels,nightVisits)
# Concatenate the results back into a single dataframe.
objectDF = pd.concat(results)

Processed visit 0845598
Processed visit 0845602
Processed visit 0845618
Processed visit 0845612
Processed visit 0845582
Processed visit 0845614
Processed visit 0845616
Processed visit 0845590
Processed visit 0845606
Processed visit 0845604
Processed visit 0845584
Processed visit 0845592
Processed visit 0845608
Processed visit 0845588
Processed visit 0845610
Processed visit 0845600
Processed visit 0845594
Processed visit 0845596
Processed visit 0845586
Processed visit 0845580
Processed visit 0845603
Processed visit 0845599
Processed visit 0845619
Processed visit 0845613
Processed visit 0845615
Processed visit 0845591
Processed visit 0845607
Processed visit 0845617
Processed visit 0845583
Processed visit 0845611
Processed visit 0845605
Processed visit 0845593
Processed visit 0845589
Processed visit 0845597
Processed visit 0845601
Processed visit 0845585
Processed visit 0845595
Processed visit 0845587
Processed visit 0845609
Processed visit 0845581
Processed visit 0845624
Processed visit 

In [78]:
timeFile='/astro/store/pogo4/smotherh/DEEP/PointingGroups/A0b/TrillingA0b_times.dat'
print(times)
visitKey,times=np.loadtxt(timeFile,unpack=True)
x2= np.array([658.212368,3857.124629])
x1 = np.array([646.777908,3818.317475])
xtimes = [0,0]
xtimes[0] = times[visitKey==int(PointingGroups[0:1]['visit_id'])][0]
xtimes[1] = times[visitKey==int(PointingGroups[-1:]['visit_id'])][0]
dt = (xtimes[1]-xtimes[0])
dx = x2-x1
speed = np.linalg.norm(dx)/dt
np.arctan2(dx[1],dx[0])+2*np.pi

[58576.21464894 58576.21635922 58576.21807104 58576.21977973
 58576.22149772 58576.22320918 58576.22491697 58576.22663981
 58576.22835088 58576.23006338 58576.23177815 58576.23349661
 58576.23523123 58576.23694325 58576.23867067 58576.24039549
 58576.24213132 58576.24384934 58576.2455702  58576.24729172
 58576.24901113 58576.25073341 58576.25245725 58576.25417461
 58576.25588848 58576.25761098 58576.25934749 58576.26106502
 58576.26277176 58576.26449399 58576.26623042 58576.26795126
 58576.26968398 58576.2714052  58576.27313266 58576.27485699
 58576.27658462 58576.27831866 58576.28004479 58576.28176347
 58576.2834855  58576.28519258 58576.28692197 58576.28864878
 58576.29036216 58576.29207627 58576.29380037 58576.29554184
 58576.29728419 58576.29900413 58576.30072641 58576.30244478
 58576.30415856 58576.3058834  58576.30759803 58576.30933378
 58576.31106413 58576.31280495 58576.31454319 58576.31627019
 58576.31798933 58576.31970708 58576.32143029 58576.3231561
 58576.32487253 58576.326

7.567441905268826

In [77]:
objectDF.query('ccd>-1')

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
3,0845581,2013 GT136,14.0,25.0,55.2,-13.0,49.0,47.0,24.4,-2.0,0.0,A0b,43,658.212368,3857.124629,2.0
18,0845581,2013 GT136,14.0,25.0,54.5,-13.0,49.0,44.0,24.4,-2.0,0.0,A0b,43,646.777908,3818.317475,2.0
3,0845581,2013 GT136,14.0,25.0,55.2,-13.0,49.0,47.0,24.4,-2.0,0.0,A0b,43,658.212368,3857.124629,2.0
18,0845581,2013 GT136,14.0,25.0,54.5,-13.0,49.0,44.0,24.4,-2.0,0.0,A0b,43,646.777908,3818.317475,2.0
3,0845583,2013 GT136,14.0,25.0,55.2,-13.0,49.0,47.0,24.4,-2.0,0.0,A0b,43,658.212368,3857.124629,2.0
18,0845583,2013 GT136,14.0,25.0,54.5,-13.0,49.0,44.0,24.4,-2.0,0.0,A0b,43,646.777908,3818.317475,2.0
3,0845583,2013 GT136,14.0,25.0,55.2,-13.0,49.0,47.0,24.4,-2.0,0.0,A0b,43,658.212368,3857.124629,2.0
18,0845583,2013 GT136,14.0,25.0,54.5,-13.0,49.0,44.0,24.4,-2.0,0.0,A0b,43,646.777908,3818.317475,2.0
3,0845585,2013 GT136,14.0,25.0,55.2,-13.0,49.0,47.0,24.4,-2.0,0.0,A0b,43,658.212368,3857.124629,2.0
18,0845585,2013 GT136,14.0,25.0,54.5,-13.0,49.0,44.0,24.4,-2.0,0.0,A0b,43,646.777908,3818.317475,2.0


In [46]:
objectDF = objectDF.query('ccd>-1')
filepath= '/astro/store/pogo4/smotherh/DEEP/PointingGroups/A0b/warps'
#filepath = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/Pointing_Group_{0:03}/warps/{0:03}'
# Pointing group number. See PickledPointings.pkl for more info
#pgNum=303

# Generate a list of names and sort by magnitude
cutOL = objectDF.drop_duplicates('name')
cutOL = cutOL.sort_values('v_mag')

for name in cutOL['name'].get_values():
    makeStamps(name,objectDF,filepath,stampSize=[21,21],numCols=5,timeFile='/astro/store/pogo4/smotherh/DEEP/PointingGroups/A0b/TrillingA0b_times.dat')

KeyError: Index(['name'], dtype='object')

In [7]:
timeFile='/astro/store/pogo4/smotherh/DEEP/PointingGroups/A0b/TrillingA0b_times.dat'
visitKey,times=np.loadtxt(timeFile,unpack=True)
dt=times[visitKey==visitId[-1]]-times[visitKey==visitId[0]]
dt=dt[0]

NameError: name 'visitId' is not defined

In [ ]:
def makeStamps(name,objectList,imagePath,imagePlane='science',numCols=5,
               stampSize=[31,31],timeFile='/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/loriallen_times.dat'):
    """Generate postage stamps of an MPC object in the Lori Allen Dataset.
    
    INPUT-
        name: This is the name of the object for which to find the image.
            Names come from the query_MPC notebook that pulls data down from
            the Minor Planets Center.

        objectList: A pandas dataframe as generated by query_MPC.

        imagePath: The path to the stack of images from which to make stamps.

        numCols: The number of columns in the postage stamp subplot.

        imagePlane : From which plane of the fits file should the stamps be
            made? Acceptable options:
            'science' : The science image plane
            'mask' : The mask image plane
            'variance' : The varience image plane
    """
    # Set the plane number used for loading the data from a fits file
    if imagePlane == 'science':
        imagePlaneNum = 1
    elif imagePlane == 'mask':
        imagePlaneNum = 2
    elif imagePlane == 'variance':
        imagePlaneNum = 3
    # Make a dataframe of a single object so that we can plot the stamps for only this object
    singleObject = objectList[objectList['name']==name]
    singleObject.reset_index(inplace=True)

    # Find the number of subplots to make. Add one for the coadd.
    numPlots = len(singleObject.index)+1
    # Compute number of rows for the plot
    numRows = numPlots // numCols
    # Add a row if numCols doesn't divide evenly into numPlots
    if (numPlots % numCols):
        numRows+=1
    # Add a row if numRows=1. Avoids an error caused by ax being 1D.
    if (numRows==1):
        numRows+=1
    # Generate the subplots, setting the size with figsize
    fig,ax = plt.subplots(nrows=numRows,ncols=numCols,
                          figsize=[3*numCols,3.5*numRows])
    objectMag = np.max(singleObject['v_mag'])
    # Find object velocity in pixels/day and the object angle in radians
    # total_motion is in arcsec/hr. DECam has .26arcsec/pixel ccd's. 24 hr/day.
    # Load initial and final object positions and calculate the trajectory angle
    findMotion = singleObject.query('ccd=={}'.format(singleObject['ccd'].values[0]))
    xi = np.array([findMotion['x_pixel'].values[0],
                   findMotion['y_pixel'].values[0]])
    xf = np.array([findMotion['x_pixel'].values[-1],
                   findMotion['y_pixel'].values[-1]])
    dx = xf-xi
    objectAngle = np.arctan2(dx[1],dx[0])
    dr = np.linalg.norm(dx)
    visitId=np.array(findMotion['visit_id'])
    visitKey,times=np.loadtxt(timeFile,unpack=True)
    dt=times[visitKey==visitId[-1]]-times[visitKey==visitId[0]]
    dt=dt[0]
    objectVel = dr/dt
    xVel = dx[0]/dt
    yVel = dx[1]/dt
    
    if objectAngle<0:
        objectAngle += 2*np.pi
    figTitle = '{}: {} image\nv_mag={}, velocity=[{:.2f},{:.2f}]={:.2f} px/day, angle={:.2f}'
    fig.suptitle(figTitle.format(name,imagePlane,objectMag,xVel,yVel,objectVel,objectAngle),
                 fontsize=16)
    # Turn off all axes. They will be turned back on for proper plots.
    for row in ax:
        for column in row:
            column.axis('off')

    stampEdge = (stampSize[0]-1)/2
    size = stampSize[0]
    x = np.linspace(-stampEdge, stampEdge, size)
    y = np.linspace(-stampEdge, stampEdge, size)
    sigma_x = 1.4
    sigma_y = 1.4

    x, y = np.meshgrid(x, y)
    gaussian_kernel = (1/(2*np.pi*sigma_x*sigma_y) 
        * np.exp(-(x**2/(2*sigma_x**2) + y**2/(2*sigma_y**2))))
    sum_pipi = np.sum(gaussian_kernel**2)
    noise_kernel = np.zeros(stampSize)
    x_mask = np.logical_or(x>5, x<-5)
    y_mask = np.logical_or(y>5, y<-5)
    mask = np.logical_or(x_mask,y_mask)
    noise_kernel[mask] = 1

    # Set the axis indexes. These are needed to plot the stamp in the correct subplot
    axi=0
    axj=1
    for i,row in singleObject.iterrows():
        # Get the Lori Allen visit id from the single object list
        visit_id = row['visit_id']
        ccd = row['ccd']
        # Get the x and y values from the first object in the cut list. Round to an integer.
        objectLoc = np.round([row['x_pixel'],row['y_pixel']])
        # Open up the fits file of interest using the pre-defined filepath string
        hdul = fits.open(os.path.join(imagePath,'{:02}/{}.fits'.format(ccd,visit_id)))

        # Generate the minimum and maximum pixel values for the stamps using stampSize
        xmin = int(objectLoc[0]-(stampSize[0]-1)/2+0.5)-1
        xmax = int(objectLoc[0]+(stampSize[0]-1)/2+0.5)
        ymin = int(objectLoc[1]-(stampSize[1]-1)/2+0.5)-1
        ymax = int(objectLoc[1]+(stampSize[1]-1)/2+0.5)

        im_dims = np.shape(hdul[imagePlaneNum].data)
        # Plot the stamp
        stampData = hdul[imagePlaneNum].data[ymin:ymax,xmin:xmax]
        #print(np.isnan(stampData))
        stampData[np.isnan(stampData)] = 0.0
        if i==0:
            coaddData=stampData
        else:
            coaddData+=stampData
        im = ax[axi,axj].imshow(stampData,cmap=plt.cm.bone)
        signal = np.sum(stampData*gaussian_kernel)
        noise = np.var(stampData*noise_kernel)
        SNR = signal/np.sqrt(noise*sum_pipi)
        ax[axi,axj].set_title(
            'ccd={} | visit={}\nSNR={:.2f}'.format(ccd,visit_id,SNR))
        ax[axi,axj].axis('on')
        # Compute the axis indexes for the next iteration
        if axj<numCols-1:
            axj+=1
        else:
            axj=0
            axi+=1
    im = ax[0,0].imshow(coaddData,cmap=plt.cm.bone)
    signal = np.sum(coaddData*gaussian_kernel)
    noise = np.var(coaddData*noise_kernel)
    SNR = signal/np.sqrt(noise*sum_pipi)
    ax[0,0].axis('on')
    _=ax[0,0].set_title('Coadd | SNR={:.2f}'.format(SNR))